
**Final Project**

Kim Jin (hj1314@nyu.edu)

11/26/2024

In [ ]:
import numpy as np
import pandas as pd
import scipy
import stat
import matplotlib.pyplot as plt
import os
import re
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Notes

In [ ]:
https://www.nyc.gov/site/nypd/stats/traffic-data/traffic-data-collision.page

In [ ]:
## Load the files
## skip some rows to exclude the notes.
staten_island = pd.read_excel('sihacc.xlsx', sheet_name=None, skiprows=4)
queens = pd.read_excel('qnhacc.xlsx', sheet_name=None, skiprows=4)
brooklyn = pd.read_excel('bkhacc.xlsx', sheet_name=None, skiprows=4)
bronx = pd.read_excel('bxhacc.xlsx', sheet_name=None, skiprows=4)
manhattan = pd.read_excel('mnhacc.xlsx', sheet_name=None, skiprows=4)
staten_island_i = pd.read_excel('siacc.xlsx', sheet_name=None, skiprows=3)
queens_i = pd.read_excel('qnacc.xlsx', sheet_name=None, skiprows=3)
brooklyn_i = pd.read_excel('bkacc.xlsx', sheet_name=None, skiprows=3)
bronx_i = pd.read_excel('bxacc.xlsx', sheet_name=None, skiprows=3)
manhattan_i = pd.read_excel('mnacc.xlsx', sheet_name=None, skiprows=3)


#Data Cleaning

In [ ]:
## also drop the last 6 rows for they are notes
all_dictionaries = {
    "staten_island": staten_island,
    "queens": queens,
    "brooklyn": brooklyn,
    "bronx": bronx,
    "manhattan": manhattan,
    "staten_island_i": staten_island_i,
    "queens_i": queens_i,
    "brooklyn_i": brooklyn_i,
    "bronx_i": bronx_i,
    "manhattan_i": manhattan_i,
}

# Function to drop the last 6 rows for each sheet in a dictionary
def drop_last_rows(data_dict, n=6):
    for sheet_name, df in data_dict.items():
        data_dict[sheet_name] = df.iloc[:-n]  # Retain all rows except the last `n`

# Apply the function to all dictionaries
for data_dict in all_dictionaries.values():
    drop_last_rows(data_dict)

In [ ]:
## fix typo in contributing-factor sheet:ColllisionKey

all_dictionaries = {
    "staten_island": staten_island,
    "queens": queens,
    "brooklyn": brooklyn,
    "bronx": bronx,
    "manhattan": manhattan,
    "staten_island_i": staten_island_i,
    "queens_i": queens_i,
    "brooklyn_i": brooklyn_i,
    "bronx_i": bronx_i,
    "manhattan_i": manhattan_i,
}

def rename_colllisionkey_to_collisionkey(data_dict):
    for sheet_name, df in data_dict.items():
        df.rename(columns={"ColllisionKey": "CollisionKey"}, inplace=True)

for data_dict in all_dictionaries.values():
    rename_colllisionkey_to_collisionkey(data_dict)


In [ ]:
## Convert dtype of 'CollisionID', 'ColllisionKey' to str
def convert_columns_to_string(data_dict, columns):
    for sheet_name, df in data_dict.items():
        for column in columns:
            if column in df.columns:
                df[column] = df[column].astype(str)  # Convert to string
    return data_dict

all_dictionaries = [
    staten_island, queens, brooklyn, bronx, manhattan,
    staten_island_i, queens_i, brooklyn_i, bronx_i, manhattan_i
]

columns_to_convert = ['CollisionID', 'ColllisionKey']

all_dictionaries = [
    convert_columns_to_string(data_dict, columns_to_convert) for data_dict in all_dictionaries
]


- merge in three ways: injured/death counts, factors, and all
- add roadway type code and borough
- drop OccurrencePrecinctCode, RoadwayReferenceMarker, RoadwayReferenceMarker, RoadwayLocationDescription

In [ ]:
## add column: Borough

all_dictionaries = {
    "staten_island": staten_island,
    "queens": queens,
    "brooklyn": brooklyn,
    "bronx": bronx,
    "manhattan": manhattan,
    "staten_island_i": staten_island_i,
    "queens_i": queens_i,
    "brooklyn_i": brooklyn_i,
    "bronx_i": bronx_i,
    "manhattan_i": manhattan_i,
}

def add_borough_column_after_collisionkey(data_dict, name):
    borough_name = re.sub(r'_i$', '', name)
    for sheet_name, df in data_dict.items():
        collision_key_idx = df.columns.get_loc('CollisionKey') + 1
        df.insert(collision_key_idx, 'Borough', borough_name)

for dict_name, data_dict in all_dictionaries.items():
    add_borough_column_after_collisionkey(data_dict, dict_name)


In [ ]:
## Now, RoadType
## For intersection, just I
## For RoadwayType, change into RoadType and so on
def add_roadtype_column_after_borough(data_dict):
    for sheet_name, df in data_dict.items():
        borough_idx = df.columns.get_loc('Borough') + 1
        df.insert(borough_idx, 'RoadType', "I")

for dict_name, data_dict in all_dictionaries.items():
    if dict_name.endswith("_i"):
        add_roadtype_column_after_borough(data_dict)

In [ ]:
## A bit tricky part:
## first change the column name from RoadwayTypeCode to RoadType for insistent
def rename_roadwaytypecode_to_roadtype(data_dict):
    data_dict['RoadwayCollisions'].rename(columns={'RoadwayTypeCode': 'RoadType'}, inplace=True)

for dict_name, data_dict in all_dictionaries.items():
    if not dict_name.endswith("_i"):
        rename_roadwaytypecode_to_roadtype(data_dict)

In [ ]:
## Add RoadType column to ContrFactor sheet
def add_roadtype_to_factors(data_dict):
    collisions_df = data_dict['RoadwayCollisions']
    factors_df = data_dict['RoadwayVehiclesContrFactors']

    roadtype_mapping = collisions_df[['CollisionID', 'RoadType']].drop_duplicates().set_index('CollisionID')['RoadType']

    factors_df['RoadType'] = factors_df['CollisionID'].map(roadtype_mapping)

    borough_idx = factors_df.columns.get_loc('Borough') + 1
    columns = list(factors_df.columns)
    columns.remove('RoadType')
    columns.insert(borough_idx, 'RoadType')
    data_dict['RoadwayVehiclesContrFactors'] = factors_df[columns]

for dict_name, data_dict in all_dictionaries.items():
    if not dict_name.endswith("_i"):
        add_roadtype_to_factors(data_dict)

In [ ]:
## Now let's start merging
## Again, I plan to merge and create 3 dfs
## One for each sheet, then everything
## But, multicollinearity and sparcity will be issues
## since we have lots of one-hot encoding
## Here I'll also drop some columns that I really don't think we need
## Let's start with Collisions
## I drops some address columns, and only keep Address, which is RoadwayName and IntersectionAddress
## and for Intersection sheet I change collision_at_intersection to collision_at_address for consistency

def drop_repeated_columns(data_dict):
    collisions_df = data_dict['RoadwayCollisions']

    collisions_df.rename(columns={'RoadwayName': 'Address'}, inplace=True)
    data_dict['RoadwayCollisions'].drop(
        columns=['RoadwayDirection', 'RoadwayLocationDescription', 'RoadwayReferenceMarker'],
        inplace=True,
        errors='ignore'
    )

    data_dict['RoadwayCollisions'] = collisions_df

for dict_name, data_dict in all_dictionaries.items():
    if not dict_name.endswith("_i"):
        drop_repeated_columns(data_dict)

In [ ]:
def update_columns_for_i(data_dict):
    for sheet_name, df in data_dict.items():
        df.drop(columns=['IntersectingStreet', 'CrossStreet'], inplace=True, errors='ignore')

        df.rename(
            columns={
                'Collision_ at_Intersection': 'Collision_ at_Location',
                'IntersectionAddress': 'Address'
            },
            inplace=True
        )

for dict_name, data_dict in all_dictionaries.items():
    if dict_name.endswith("_i"):
        update_columns_for_i(data_dict)

In [ ]:
# Function to check column name consistency between IntersectCollisions and RoadwayCollisions
def check_column_consistency(dictionaries):
    i_columns = set()
    for dict_name, data_dict in dictionaries.items():
        if dict_name.endswith("_i") and 'IntersectCollisions' in data_dict:
            i_columns = set(data_dict['IntersectCollisions'].columns)
            break

    non_i_columns = set()
    for dict_name, data_dict in dictionaries.items():
        if not dict_name.endswith("_i") and 'RoadwayCollisions' in data_dict:
            non_i_columns = set(data_dict['RoadwayCollisions'].columns)
            break

    same_columns = i_columns == non_i_columns
    print(f"Do IntersectCollisions (_i) and RoadwayCollisions (non-_i) have the same columns? {same_columns}")

    if not same_columns:
        print("Columns in IntersectCollisions (_i) but not in RoadwayCollisions (non-_i):")
        print(i_columns - non_i_columns)
        print("Columns in RoadwayCollisions (non-_i) but not in IntersectCollisions (_i):")
        print(non_i_columns - i_columns)

check_column_consistency(all_dictionaries)

Do IntersectCollisions (_i) and RoadwayCollisions (non-_i) have the same columns? True


In [ ]:
roadway_collisions_list = []
intersect_collisions_list = []

for dict_name, data_dict in all_dictionaries.items():
    if not dict_name.endswith("_i"):
        roadway_collisions_list.append(data_dict['RoadwayCollisions'])

for dict_name, data_dict in all_dictionaries.items():
    if dict_name.endswith("_i"):
        intersect_collisions_list.append(data_dict['IntersectCollisions'])

collision_counts = pd.concat(roadway_collisions_list + intersect_collisions_list, ignore_index=True)

In [ ]:
collision_counts['CollisionID'] = collision_counts['CollisionID'].astype(str).str.replace(r'\.0$', '', regex=True)
collision_counts['CollisionKey'] = collision_counts['CollisionKey'].astype(str).str.replace(r'\.0$', '', regex=True)

In [ ]:
# collision_counts.to_csv("collision_counts.csv", index=True)

collision_counts is the df that conbine all sheets that include collision counts.


In [ ]:
## Now let's merge the second sheet
## first drop column RoadwayReferenceMarker so all have the same structure
def drop_column_roadway_reference_marker(data_dict):
    data_dict['RoadwayVehiclesContrFactors'].drop(
        columns=['RoadwayReferenceMarker'],
        inplace=True,
        errors='ignore'
    )

for dict_name, data_dict in all_dictionaries.items():
    if not dict_name.endswith("_i"):
        drop_column_roadway_reference_marker(data_dict)

In [ ]:
roadway_vehicles_list = []
intersect_vehicles_list = []

for dict_name, data_dict in all_dictionaries.items():
    if not dict_name.endswith("_i"):
        roadway_vehicles_list.append(data_dict['RoadwayVehiclesContrFactors'])

for dict_name, data_dict in all_dictionaries.items():
    if dict_name.endswith("_i"):
        intersect_vehicles_list.append(data_dict['IntersectVehiclesContrFactors'])

factors = pd.concat(roadway_vehicles_list + intersect_vehicles_list, ignore_index=True)

In [ ]:
factors['CollisionID'] = factors['CollisionID'].astype(str).str.replace(r'\.0$', '', regex=True)
factors['CollisionKey'] = factors['CollisionKey'].astype(str).str.replace(r'\.0$', '', regex=True)

In [ ]:
# factors.to_csv("factors.csv", index=True)

In [ ]:
## now, everything together
## CollisionID as the primary key
df = factors.merge(collision_counts, on='CollisionID', how='left')
df = df.loc[:, ~df.columns.duplicated()]
print(df.columns.tolist())

['CollisionID', 'CollisionKey_x', 'Borough_x', 'RoadType_x', 'VehicleSequenceNumber', 'VehicleTypeCode', 'VehicleTypeDescription', 'ContributingFactorCode', 'ContributingFactorDescription', 'OccurrencePrecinctCode', 'CollisionKey_y', 'Borough_y', 'RoadType_y', 'Collision_ at_Location', 'Address', 'CollisionVehicleCount', 'CollisionInjuredCount', 'CollisionKilledCount', 'Vehicles_or_MotoristsInvolved', 'PersonsInjured', 'PersonsKilled', 'MotoristsInjured', 'MotoristsKilled', 'PassengInjured', 'PassengKilled', 'CyclistsInjured', 'CyclistsKilled', 'PedestrInjured', 'PedestrKilled', 'Injury_or_Fatal', 'Bicycle']


In [ ]:
df = df.drop(columns=['CollisionKey_y', 'Borough_y', 'RoadType_y'])
df = df.rename(columns=lambda col: col.replace('_x', '') if col.endswith('_x') else col)
df['CollisionKey'] = df['CollisionKey'].astype(str).str.replace(r'\.0$', '', regex=True)

In [ ]:
# df.to_csv("df.csv", index=True)

Huizhen Jin's note: Be careful when using df: mass multicolinearity. As we know, one incedent can involve multiple parties but in data have the same injuries or death count, so it can be not accurate. For example, for one incident, a bus hit a bike; the bus is accounted for most of the counts, but in df it will show the same for bikc. So, really be careful. I think using factors and collision_counts are mostly enough, but just in case, here is a total df. Tell me if anything needed!